In [742]:
import pandas as pd

In [743]:
df=pd.read_csv('fake_news.csv',on_bad_lines='skip',engine='python')

In [744]:
[df['label'].unique()] #found out that label column has these 2 outliers later lol

[array(['1', '0', None, ' как люди воспринимают своё положение.',
        ' чтобы это была дорога с двусторонним движением.'], dtype=object)]

In [745]:
df = df[df['label'].isin(['0', '1'])]
df = df.reset_index(drop=True)

In [746]:
df['label'].unique()

array(['1', '0'], dtype=object)

In [747]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20793,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20794,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20795,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20796,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [748]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [749]:
df=df.dropna(axis=0) #for dropping all rows

In [750]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [751]:
df.shape

(18283, 5)

In [752]:
x=df.drop(labels='label',axis=1)

In [753]:
y=df['label']

In [754]:
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20793,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20794,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20795,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20796,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [755]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20793    0
20794    0
20795    0
20796    1
20797    1
Name: label, Length: 18283, dtype: object

In [756]:
import tensorflow as tf

In [757]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Dropout
from tensorflow.keras.models import Sequential

In [758]:
vocabulary_size=10000

In [759]:
messages=x.copy()

In [760]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20793,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20794,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20795,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20796,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [761]:
messages.reset_index(inplace=True) #as in the above ex 6 is missing, there might be more missing further down too

In [762]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18278,20793,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18279,20794,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18280,20795,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18281,20796,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [763]:
import nltk
import re
from nltk.corpus import stopwords

In [764]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\swapn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [765]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

applying both stopwords and lemmatization

In [766]:
corpus=[]
stop_words=stopwords.words('english')
for i in range(0,len(messages['title'])):
    review=re.sub('[^a-zA-Z0-9]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[lemmatizer.lemmatize(word) for word in review if not word in stop_words]
    review=' '.join(review)
    corpus.append(review)

In [767]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 '15 civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor p

now doing OHE

In [768]:
ohe_repr=[one_hot(word,vocabulary_size) for word in corpus]

In [769]:
(ohe_repr)

[[5192, 9838, 4808, 3720, 8641, 9318, 3676, 4003, 5412, 9908],
 [7267, 939, 7956, 5620, 307, 1657, 6385],
 [1833, 1011, 7244, 8868],
 [8069, 7430, 8602, 7575, 554, 6758, 1792],
 [5952, 307, 4144, 5959, 5742, 6706, 307, 1049, 6773, 3768],
 [2593,
  3580,
  5491,
  5338,
  3300,
  3234,
  9204,
  6277,
  4893,
  6430,
  4934,
  3634,
  2337,
  9275,
  6385],
 [6551, 1401, 9008, 9553, 1928, 670, 9263, 3967, 2398, 2779, 3925],
 [6297, 4631, 4584, 1630, 5491, 6515, 3234, 1567, 2398, 2779, 3925],
 [1261, 9398, 2722, 3763, 5367, 2783, 4142, 6066, 3234, 5268],
 [6505, 6712, 9334, 5341, 3242, 2606, 418, 3640],
 [3809, 2313, 81, 2967, 2110, 6530, 9934, 2962, 5997, 7796, 5019],
 [554, 5729, 8641, 2992, 3234, 5491],
 [7771, 7540, 1482, 193, 5839, 6743, 4576, 9436, 4055],
 [4833, 2901, 2599, 9346, 6400, 8819, 2846, 2398, 2779, 3925],
 [8878, 5475, 6919, 8280, 5634, 2398, 2779, 3925],
 [2179, 1270, 3488, 658, 1628, 4918, 1472, 4098, 8959, 9833, 9653, 6306],
 [5263, 7222, 939],
 [3652, 5232, 1776, 86

In [770]:
max(len(i) for i in ohe_repr) #we got the max length of a word as 47

47

Applying padding now

In [771]:
sentence_length=47

In [772]:
embedded_docs=pad_sequences(ohe_repr,padding='pre',maxlen=sentence_length)

In [773]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 5192, 9838, 4808, 3720, 8641, 9318, 3676,
       4003, 5412, 9908], dtype=int32)

applying embedding layer and LSTM

In [774]:
embedding_vector_size=100 #this means like each entry in embedded_docs[0] and the rest will be represented with a vector of size 100, like 0 will be represented with a size of 100, 5192 with a size of 100 and the rest goes on

In [775]:
model=Sequential([Input(shape=(sentence_length,))])

In [776]:
model.add(Embedding(input_dim=vocabulary_size,output_dim=embedding_vector_size))

In [777]:
model.add(Dropout(0.3)) #basically hiding 30% of the data while training to help decrease over-fitting

In [778]:
model.add(LSTM(100)) #it's the number of neurons

In [ ]:
model.add(Dropout(0.3)) #doing the same after the LSTM, basically dropout like hides 30% of the data (in my case) in each batch of training, and in each batch it's a diff 30% of the data, which keeps rotating

In [780]:
model.add(Dense(1,activation='sigmoid')) #as this is binary classification

In [781]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [782]:
print(model.summary())

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_12 (Embedding)        │ (None, 47, 100)        │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 47, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,080,501 (4.12 MB)

 Trainable params: 1,080,501 (4.12 MB)

 Non-trainable params: 0 (0.00 B)

None


In [783]:
from sklearn.preprocessing import LabelEncoder

In [784]:
y=LabelEncoder().fit_transform(y)

In [785]:
len(embedded_docs),y.shape

(18283, (18283,))

In [786]:
import numpy as np

In [787]:
x=np.array(embedded_docs)

In [788]:
y=np.array(y)

In [789]:
x.shape,y.shape

((18283, 47), (18283,))

In [790]:
from sklearn.model_selection import train_test_split

In [791]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.33)

In [792]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=5,batch_size=128)

Epoch 1/5
96/96 ━━━━━━━━━━━━━━━━━━━━ 12s 110ms/step - accuracy: 0.8535 - loss: 0.3342 - val_accuracy: 0.9055 - val_loss: 0.2106
Epoch 2/5
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step - accuracy: 0.9403 - loss: 0.1444 - val_accuracy: 0.9244 - val_loss: 0.1823
Epoch 3/5
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 105ms/step - accuracy: 0.9683 - loss: 0.0873 - val_accuracy: 0.9180 - val_loss: 0.1991
Epoch 4/5
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 102ms/step - accuracy: 0.9841 - loss: 0.0498 - val_accuracy: 0.9148 - val_loss: 0.2455
Epoch 5/5
96/96 ━━━━━━━━━━━━━━━━━━━━ 10s 102ms/step - accuracy: 0.9902 - loss: 0.0313 - val_accuracy: 0.9158 - val_loss: 0.2737


In [793]:
y_pred=model.predict(x_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [794]:
y_pred=np.where(y_pred > 0.5, 1,0)

In [795]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [796]:
print(confusion_matrix(y_test,y_pred)) #this is remapping my values to 1 if y_pred >0.5 (fake news) or 0 otherwise (true)

[[3141  274]
 [ 234 2385]]


In [797]:
print(accuracy_score(y_test,y_pred))

0.9158104076897581


In [798]:
from sklearn.metrics import classification_report

In [799]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      3415
           1       0.90      0.91      0.90      2619

    accuracy                           0.92      6034
   macro avg       0.91      0.92      0.91      6034
weighted avg       0.92      0.92      0.92      6034

